In [1]:
import pandas as pd
import numpy as np
import tqdm
import datetime
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

from scipy import sparse

pd.set_option("max.columns", None)

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv("raw_data_all_labeled2.csv", index_col=0).dropna(subset=['y'])
df.shape

(1410, 15)

In [3]:
df.duplicated().sum()

0

In [4]:
df.duplicated(['title']).sum()

0

### 1. Limpeza da data

In [5]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['title']
df_limpo['date'] = pd.to_datetime(df['upload_date'])

### 2. Limpeza de views

In [6]:
views = df['view_count'].fillna(0)
df_limpo['views'] = views

### 3. Features 

In [7]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()
today = datetime.datetime.today().strftime("%Y-%m-%d")
features['tempo_desde_pub'] = (pd.to_datetime(today) -  df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = (features['views'] / features['tempo_desde_pub']).round(3)
features.drop(['tempo_desde_pub'], axis=1, inplace=True)

In [8]:
mask_train = df_limpo['date'] < '2020-03-10'
mask_val = df_limpo['date'] >= '2020-03-10'

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((714, 2), (696, 2), (714,), (696,))

In [9]:
title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

# Min df - minimo de vezes que palavra tem que aparecer pra virar coluna
title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,2))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [10]:
title_bow_train.shape

(714, 1134)

In [11]:
# Concatenando as variávels numéricas com as geradas pelo TfidfVectorizer
Xtrain_wtitle = sparse.hstack([Xtrain, title_bow_train])
Xval_wtitle = sparse.hstack([Xval, title_bow_val])
Xtrain_wtitle.shape, Xval_wtitle.shape

((714, 1136), (696, 1136))

In [12]:
mdl = RandomForestClassifier(n_estimators=1000, min_samples_leaf=1, random_state=0, class_weight='balanced', n_jobs=4)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=4,
                       random_state=0)

In [13]:
p = mdl.predict_proba(Xval_wtitle)[: ,1]

In [14]:
metrics.average_precision_score(yval, p), metrics.roc_auc_score(yval, p)

(0.3969764593456903, 0.677681712479876)

### 5. LightGBM

In [15]:
from lightgbm import LGBMClassifier

In [16]:
mdl = LGBMClassifier(random_state=0, class_weight='balanced', n_jobs=4)
mdl.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(class_weight='balanced', n_jobs=4, random_state=0)

In [17]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

/home/cesar/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [18]:
metrics.average_precision_score(yval, p), metrics.roc_auc_score(yval, p)

(0.34044253688199166, 0.6430922425615646)

### 6. Bayesian Optimization

In [19]:
from skopt import forest_minimize

In [20]:
results = []
def tune_lgbm(params):
    tunning = {}
    print()
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    min_df = params[6]
    ngram_range = (1, params[7])
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)

    Xtrain_wtitle = sparse.hstack([Xtrain, title_bow_train])
    Xval_wtitle = sparse.hstack([Xval, title_bow_val])

    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth,
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1, n_estimators=n_estimators,
                        random_state=0, class_weight='balanced', n_jobs=4)
    mdl.fit(Xtrain_wtitle, ytrain)
   
    pred = mdl.predict_proba(Xval_wtitle)[:, 1]
    print(metrics.roc_auc_score(yval, pred))
    tunning['params'] = params
    tunning['roc'] = metrics.roc_auc_score(yval, pred)
    tunning['avg_prec'] = metrics.average_precision_score(yval, pred)
    results.append(tunning)
    return -metrics.average_precision_score(yval, pred)


In [21]:
space = [(1e-3, 1e-1, 'log-uniform'), #lr
         (1,10), #max_depth
         (1,20), #min_child_samples
         (0.05, 1.), #subsample
         (0.05, 1.), #colsample_bytree
         (100, 1000), #n_estimators
         (1,5), # min_df
         (1,5)] # ngram_range

In [22]:
res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.

[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]


/home/cesar/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6575696142150139
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.3110
Function value obtained: -0.3645
Current minimum: -0.3645
Iteration No: 2 started. Evaluating function at random point.

[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]


/home/cesar/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.644958559418043
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.2844
Function value obtained: -0.3450
Current minimum: -0.3645
Iteration No: 3 started. Evaluating function at random point.

[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]


...

0.645650229562936
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 0.4809
Function value obtained: -0.3484
Current minimum: -0.4003
Iteration No: 49 started. Searching for the next optimal point.

[0.053539787146168444, 7, 20, 0.9806497569171833, 0.0985142521272917, 290, 3, 1]
0.6529306541053008


/home/cesar/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 0.4117
Function value obtained: -0.3795
Current minimum: -0.4003
Iteration No: 50 started. Searching for the next optimal point.

[0.03137052946911072, 9, 20, 0.6125784338354515, 0.08679440024036054, 728, 1, 1]


/home/cesar/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:497: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6151332657563651
Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.5825
Function value obtained: -0.3441
Current minimum: -0.4003


In [23]:
sorted(results, key = lambda i: i['avg_prec'],reverse=True)[0]

{'params': [0.001862037165403463,
  10,
  20,
  0.7594594688994479,
  0.06663099351693305,
  118,
  3,
  1],
 'roc': 0.678778844433844,
 'avg_prec': 0.4003489238281557}

### 7. Logistic Reg

In [26]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler

In [51]:
Xtrain_wtitle2 = sparse.csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = sparse.csr_matrix(Xval_wtitle.copy())

#scaler = StandardScaler()
scaler = MaxAbsScaler()

# Xtrain_wtitle2[: , :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
# Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())

Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
Xval_wtitle2 = scaler.transform(Xval_wtitle2)

In [52]:
Xval_wtitle2.shape

(696, 1136)

In [56]:
mdl = LogisticRegression(C=0.5, n_jobs=4, random_state=4)
mdl.fit(Xtrain_wtitle2, ytrain)

LogisticRegression(C=0.5, n_jobs=4, random_state=4)

In [57]:
p = mdl.predict_proba(Xval_wtitle2)[:, 1]

In [58]:
metrics.average_precision_score(yval, p), metrics.roc_auc_score(yval, p)

(0.41472090277819385, 0.6588873650945083)

In [59]:
# (0.4043414314912761, 0.6789338739490788) - sem tunning StandardScaler
# (0.3988238048468208, 0.6462226462345716) - sem tunning MaxAbScaler
# (0.33826219541849384, 0.6082881163913899) - C=10, MaxAbScaler
# (0.41472090277819385, 0.6588873650945083) - C=0.5 MaxAbScaler